## Script to filter processed ILATM2 data within 5km radius of PROMICE and GC-NET on-ice weather stations. 

### - Shapefiles for weather station locations and coastline taken from QGreenland. 

In [ ]:
# Imports and dependencies 

import pandas as pd
import geopandas as gpd
import os

In [ ]:
coastline_gpd = gpd.read_file("./greenlandCoastline.shp") # Load coastline .shp 
GCNET_gdf = gpd.read_file("./WxStations/GCNETSites.shp") # Load weather station .shps 
PROMICE_gdf = gpd.read_file("./WxStations/PROMICESites.shp")

GCNET_gdf = GCNET_gdf.to_crs(coastline_gpd.crs) # Set weather stations to coastline CRS 
PROMICE_gdf = PROMICE_gdf.to_crs(coastline_gpd.crs)

GCNET_gdf["buffered_5km"] = GCNET_gdf.buffer(5000) # Buffer station locations with a 5 km radius 
PROMICE_gdf["buffered_5km"] = PROMICE_gdf.buffer(5000)

weather_stations = gpd.GeoDataFrame(pd.concat([GCNET_gdf, PROMICE_gdf], ignore_index=True), crs=coastline_gpd.crs) # Concatenate 
 
main_folder_path = "./weather_stations/"   
output_folder_path = "./weather_stations_5km/"

os.makedirs(output_folder_path, exist_ok=True) # Create output directory if it doesn't exist

In [ ]:
# Process each buffered .csv

for csv_file in os.listdir(main_folder_path):
    if csv_file.endswith('.csv'):
        csv_file_path = os.path.join(main_folder_path, csv_file)
        print(f"Processing file: {csv_file_path}")
        df = pd.read_csv(csv_file_path) # Read .csvs 
        df = df.apply(pd.to_numeric, errors='coerce').dropna()
        df = df.rename(str.strip, axis='columns')
        tmp_gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['Longitude(deg)'], df['Latitude(deg)']), crs='EPSG:4326') # Convert to gdfs 
        tmp_gdf = tmp_gdf.to_crs(coastline_gpd.crs) # Convert gdf CRS to coastline CRS 
        clipped_data = [] 
        for i in range(len(weather_stations)):
            clipped_gdf = gpd.clip(tmp_gdf, weather_stations.loc[i, "buffered_5km"]) # Clip the data within the 5km buffers 
            if not clipped_gdf.empty:
                clipped_data.append(clipped_gdf)
        if clipped_data: # Save clipped data 
            out = pd.concat(clipped_data).drop(columns='geometry') 
            output_file_path = os.path.join(output_folder_path, csv_file.replace('.csv', '_5km_buffered.csv'))
            out.to_csv(output_file_path, index=False)
            print(f"File saved: {output_file_path}")
        else: 
            print(f"No data after clipped for file: {csv_file}") 